In [9]:
import pandas as pd
import os

fields = pd.read_json('raw/fields-2012_2025.json')
standards = pd.read_json('raw/standards-2020_2025.json')
downhill_marathons = pd.read_csv('raw/downhill-marathons.csv')

In [10]:
athlinks_slugs = ['tucson-marathon', 'sundance-to-spearfish', 'pocatello', 'leading-ladies', 'jack-and-kills', 'hawaii-bird-conservation']
raceentry_slugs = ['utah-valley-marathon']
ultrasignup_slugs = ['tunnel-vision', 'tunnel-light', 'light-at-the-end-of-the-tunnel']
chronokeep_slugs = ['super', 'cascade-express']
webscorer_slugs = ['sun', 'east-canyon']
runsignup_slugs = ['runtastic-nebo']
brooksee_slugs = ['revel-white-mountains', 'revel-mt-charleston', 'revel-big-bear', 'revel-big-cottonwood', 'deseret-news', 'bears-ears']


In [ ]:
results_df = pd.DataFrame(columns=['slug', 'year', 'time', 'gender', 'age_group'])
for year in range(2023, 2025 + 1):
    for slug in athlinks_slugs:
        results_df = pd.read_json(f'raw/marathons/{slug}-{year}.json')
        print(results_df.head(10))

FileNotFoundError: File raw/athlinks/tucson-marathon-2023.json does not exist

In [2]:
fields['"CUT-OFF TIME"*'] = pd.to_timedelta('00:0' + fields['"CUT-OFF TIME"*'])
fields[['QUALIFIERS NOT ACCEPTED', 'FIELD SIZE']] = fields[['QUALIFIERS NOT ACCEPTED', 'FIELD SIZE']].apply(lambda col: col.str.replace(',', '').astype(int))

In [3]:
standards[['MEN', 'WOMEN']] = standards[['MEN', 'WOMEN']].apply(lambda col: pd.to_timedelta(col.str.replace('hrs', 'hr')))

In [4]:
downhill_marathons['Net Downhill'] = downhill_marathons['Net Downhill'].str.replace(',', '').astype(int)
downhill_marathons['Category'] = downhill_marathons['Net Downhill'].apply(lambda x: '3,000-5,999' if 3000 <= x < 6000 else '1,500-2,999')

In [5]:
fields_to_save = fields.copy()
fields_to_save['"CUT-OFF TIME"*'] = fields_to_save['"CUT-OFF TIME"*'].apply(lambda x: x.seconds)
fields_to_save.columns = [['Year', 'Field Size', 'Cutoff Time (s)', 'Qualifiers Not Accepted']]
fields_to_save.to_csv('data/fields.csv', index=False)

standards_to_save = standards.copy()
standards_to_save[['MEN', 'WOMEN']] = standards_to_save[['MEN', 'WOMEN']].apply(lambda col: col.astype('str').apply(lambda x: x.split(' ')[2][1:]))
standards_to_save.columns = [['Age Group', 'Men', 'Women']]
standards_to_save.to_csv('data/standards.csv', index=False)

downhill_marathons_to_save = downhill_marathons.copy()
downhill_marathons_to_save.sort_values(by='Net Downhill', ascending=False, inplace=True)
downhill_marathons_to_save.columns = [['Name', 'Net Downhill (ft)', 'Year Established', 'Location', 'Category']]
downhill_marathons_to_save.to_csv('data/downhill-marathons.csv', index=False)